In [2]:
import pandas as pd
import numpy as np
from battery_plot import *

In [3]:
data = "/Users/attar/Library/CloudStorage/OneDrive-UniversityofWaterloo/Maccor Tests/JMFM_17_SOC_OCV_94plus5_Test_220624.txt"

In [4]:
df = pd.read_csv(data, skiprows=[0,1,2],sep="\t")

In [5]:
def get_sec(time_str):
    d, m, s = time_str.split(":")
    d, h = d.split("d")
    d = int(d)
    h = int(h)
    m = int(m)
    s = int(s)
    return d*86400 + h*3600 + m*60 +s

def nf_soc_compute(data, capacity, to_csv_pts = False, to_csv_curve = False):
    '''
    .txt file[, bool, bool] -> pd.DataFrame, pd.DataFrame[, .csv file, .csv file] 
    Precondition, data is a .txt file from the Maccor
    
    Parameters:
    `capacity` float
        the capacity of the LFP cell
    `to_csv_pts` bool
        to send the extracted points to a csv file
    `to_csv_curve` bool
        to send the estimated points to a csv file
    '''
    data = pd.read_csv(data, skiprows=[0,1,2], sep="\t")
    
    first_idx = data[data["MD"] == "D"].iloc[0].name
    
    df = data.loc[first_idx-1:,["Test Time", "Current", "Voltage", "MD"]].copy()
    
    df.columns = [col.lower() for col in df.columns]

    df.loc[df["md"] == "D","current"] *= -1

    delta_cap = df["current"] / 3600

    df["soc"] = list((capacity + delta_cap.cumsum()) / capacity * 100)
    df.loc[:,"test time (sec)"] = [get_sec(i) for i in df["test time"]]
    
#     if to_csv_pts:
#         name = input("Name of output file for SOC-OCV points: ")
#         data.to_csv(name)
#         print( f"\n{name} was created.\n" )
#     if to_csv_curve:
#         name = input("Name of output file for SOC-OCV curve: ")
#         curve.to_csv(name)
#         print( f"\n{name} was created.\n" )

    mask = df["test time (sec)"].diff()
    mask.iloc[0],mask.iloc[-1] = 1.0,1.0
    df["delta t"] = mask
    
    return df[["test time (sec)","delta t","current","voltage","soc"]]

In [6]:
df = nf_soc_compute(data, 18.208)

In [ ]:
train = file[["delta t","current","voltage","soc" ]].loc[:107898]

test = file[["delta t","current","voltage","soc" ]].loc[107899:]

In [ ]:
[win for win in pd.DataFrame(np.arange(100).reshape(10,10)).join(pd.Series(np.arange(10),name="10")).rolling(window = 10)]

In [6]:
df.to_csv("/Users/attar/Library/CloudStorage/OneDrive-UniversityofWaterloo/Maccor Tests/JMFM_17_SOC_OCV_94plus5_Test_220624_soc.csv", index=False)

In [ ]:
def rolling(self, df, window_size):
        '''
        implements rolling window sectioning
        There are four input features: delta_t, V, I at time t, and SOC at time t-1
        Prediction at time t uses the features given
        '''

        df_x = [window.values
                for window
                in df[["delta t","current","voltage","soc"]].iloc[:-1] #staggered left by one
                    .join(
                          df["soc"].iloc[1:] #staggered right by one
                         )
                    .rolling(window = window_size,
                            min_periods=window_size-2,
                            method = "table"
                            )][window_size:]

        df_y = [window.values
                for window
                in df["soc"].iloc[1:] #staggered right by one
                    .rolling(window = window_size,
                            min_periods=window_size-2,
                            method = "single"
                            )][window_size:]

        return np.array(df_x, dtype="float32"), np.array(df_y, dtype="float32")

def rolling_trial(self, df, window_size):
        '''
        implements rolling window sectioning
        Four input features: delta_t, I, V, SOC all at time t-1
        The prediction of SOC at time t uses no other information
        '''

        df_x = [window.values
                for window
                in df[["delta t","current","voltage","soc"]].iloc[:-1] #staggered left by one
                    .rolling(window = window_size,
                            min_periods=window_size-2,
                            method = "table"
                            )][window_size:]

        df_y = [window.values
                for window
                in df["soc"].iloc[1:] #staggered right by one
                    .rolling(window = window_size,
                            min_periods=window_size-2,
                            method = "single"
                            )][window_size:]

        return np.array(df_x, dtype="float32"), np.array(df_y, dtype="float32")

In [ ]:
data_plot(data = [df],
          x = ["test time (sec)"],
          y = [ "soc" ],
          title = "JMFM-17 SOC vs. Time",
          x_title = "Time (sec)",
          y_title = "SOC (%)",
          mode = "lines",
          color = "darkorchid",
          name = None,
          size = None
         )